### BASE DE DATOS DE SPOTIFY

---

##### Trabajo practico I - Analisis predictivo avanzado
##### *Evaluación de modelos*

### PASO 1: Importamos las librerias que utilizaremos


In [ ]:
!pip install langdetect
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=59bb94f17119830ce3d98413f47aae9405c39bfb6745597dc4436c560afee195
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import requests
import warnings
import openpyxl
import datetime
import pytz
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from langdetect import detect
from collections import defaultdict
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score, StratifiedKFold, RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, AdaBoostRegressor
import xgboost as xgb
from hyperopt import fmin, tpe, hp, Trials
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from scipy.stats import uniform,randint

### PASO 2: Cargamos el Dataset

In [ ]:
# CARGAMOS EL DATASET #
df = pd.read_csv('/content/train.csv')

# ELIMINAMOS LA COLUMNA 0 YA Q NO ES RELEVANTE A NUESTRO ANALISIS #
df = df.drop(df.columns[0], axis=1)


In [ ]:
# AGREGAR LA COLUMNA IDIOMA DE LA CANCION #
# esto tarda media hora en correr, pero corre
def detect_language(text):
    if len(text.strip()) < 3:
        return 'Desconocido'
    try:
        return detect(text)
    except:
        return 'Desconocido'
df['Idioma_Detectado'] = df['name'].apply(detect_language)

In [ ]:
# BORRAR LA COLUMNA NAME #
df.drop(columns=['name'], inplace=True)

In [ ]:
df

,artists,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,popularity,Artistas Involucrados,Idioma_Detectado
0,"['Exquisite Classic', 'Johann Sebastian Bach',...",1920,0.88600,0.2760,376000,0.2230,0,0.015800,7,0.2660,-18.500,1,0.0329,61.300,0.141,1,3,en
1,['محمد الإدريسي'],1920,0.86300,0.5030,145333,0.2820,0,0.000148,0,0.1230,-12.204,1,0.0608,131.228,0.261,0,1,ar
2,"['Exquisite Classic', 'Johann Sebastian Bach',...",1920,0.70100,0.0791,210000,0.1920,0,0.035900,2,0.0850,-17.945,0,0.0398,178.577,0.184,2,3,en
3,['Maurice Chevalier'],1920,0.99400,0.4870,191173,0.2980,0,0.000000,6,0.1330,-17.141,1,0.0980,99.989,0.738,0,1,fi
4,"['Exquisite Classic', 'Robert Schumann', 'Svia...",1920,0.99400,0.2600,312867,0.0146,0,0.728000,5,0.0875,-30.336,1,0.0408,91.646,0.118,0,3,it
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129167,['Blue Öyster Cult'],1994,0.00178,0.5160,268273,0.6390,0,0.000058,9,0.1670,-9.621,0,0.0315,134.412,0.676,34,1,en
129168,['TOTO'],1995,0.14600,0.6530,366373,0.5060,0,0.000094,10,0.0845,-8.755,1,0.0267,98.391,0.447,52,1,en
129169,['Van Morrison'],1995,0.65800,0.6600,197160,0.5410,0,0.001270,0,0.0928,-7.851,0,0.0576,93.744,0.693,63,1,en
129170,['Cocteau Twins'],1996,0.59500,0.4510,237600,0.5830,0,0.000030,4,0.1950,-6.481,0,0.0244,77.945,0.582,38,1,no


### PASO 3: particion entrenamiento y testeo

In [ ]:
y = df['popularity']
X = df.drop('popularity', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### PASO 4: armar el Pipeline

In [ ]:
class FeatureSelectionFrequencyEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, selected_features):
        self.selected_features = selected_features
        self.encoding_dict = defaultdict(int)

    def fit(self, X, y=None):
        for feature in self.selected_features:
                frequencies = X[feature].value_counts().to_dict()
                self.encoding_dict[feature] = frequencies

        return self

    def transform(self, X):
        X_copy = X.copy()
        for feature in self.selected_features:
            if feature in self.encoding_dict:
                X_copy[feature] = X_copy[feature].map(self.encoding_dict[feature])
                X_copy[feature] = X_copy[feature].fillna(1)
        return X_copy


In [ ]:
class AgregarArtistasInvolucrados(BaseEstimator, TransformerMixin):
    def __init__(self, selected_features):
        self.selected_features = selected_features

    def fit(self, X, y=None):

        return self

    def transform(self, X):
        X_copy = X.copy()
        for feature in self.selected_features:
          X_copy[feature] = X_copy[feature].str.count(',') + 1
        return X_copy

In [ ]:
nombresColumnas=df.columns.tolist()
pasos = [
    ('agregar_artistas', AgregarArtistasInvolucrados(selected_features=['artists'])),
    ('encoder', FeatureSelectionFrequencyEncoder(selected_features=['Idioma_Detectado'])),
    ('scaler', StandardScaler()),
    ('GradientBoosting', HistGradientBoostingRegressor())
]

pipe = Pipeline(pasos)

In [ ]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('agregar_artistas',
                 AgregarArtistasInvolucrados(selected_features=['artists'])),
                ('encoder',
                 FeatureSelectionFrequencyEncoder(selected_features=['Idioma_Detectado'])),
                ('scaler', StandardScaler()),
                ('GradientBoosting', HistGradientBoostingRegressor())])

### PASO 5: Competencia de modelos predictivos

##### Método 1: XGBoost

In [ ]:
pasos = [
    ('agregar_artistas', AgregarArtistasInvolucrados(selected_features=['artists'])),
    ('encoder', FeatureSelectionFrequencyEncoder(selected_features=['Idioma_Detectado'])),
    ('scaler', StandardScaler()),
    ('xgb_regressor', xgb.XGBRegressor(objective='reg:squarederror'))
]

# Create the pipeline
pipe = Pipeline(pasos)


In [ ]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

mse_scores = -cross_val_score(pipe, X, y, cv=5, scoring='neg_mean_squared_error')
r2_scores = cross_val_score(pipe, X, y, cv=5, scoring='r2')

avg_mse = mse_scores.mean()
avg_r2 = r2_scores.mean()

print(f'Promedio del Error Cuadrático Medio (MSE) con CV: {avg_mse}')
print(f'Promedio del coeficiente de determinación (R^2) con CV: {avg_r2}')

Promedio del Error Cuadrático Medio (MSE) con CV: 220.4421138860883
Promedio del coeficiente de determinación (R^2) con CV: 0.4259919588966957


##### Método 2: Hist Gradian Boosting

In [ ]:
pasos = [
        ('agregar_artistas', AgregarArtistasInvolucrados(selected_features=['artists'])),
    ('encoder', FeatureSelectionFrequencyEncoder(selected_features=['Idioma_Detectado'])),
    ('scaler', StandardScaler()),
    ('GradientBoosting', HistGradientBoostingRegressor())
]

pipe = Pipeline(pasos)

In [ ]:
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R^2): {r2}')

mse_scores = -cross_val_score(pipe, X, y, cv=5, scoring='neg_mean_squared_error')
r2_scores = cross_val_score(pipe, X, y, cv=5, scoring='r2')

avg_mse = mse_scores.mean()
avg_r2 = r2_scores.mean()

print(f'Promedio del Error Cuadrático Medio (MSE) con CV: {avg_mse}')
print(f'Promedio del coeficiente de determinación (R^2) con CV: {avg_r2}')

Mean Squared Error (MSE): 157.51756636852943
R-squared (R^2): 0.669823388441902
Promedio del Error Cuadrático Medio (MSE) con CV: 204.52617342496723
Promedio del coeficiente de determinación (R^2) con CV: 0.45302604844323646


##### Método 3: Lasso regression

In [ ]:
pasos = [
    ('agregar_artistas', AgregarArtistasInvolucrados(selected_features=['artists'])),
    ('encoder', FeatureSelectionFrequencyEncoder(selected_features=['Idioma_Detectado'])),
    ('scaler', StandardScaler()),
    ('lasso', Lasso(alpha=1.0))
]
pipe = Pipeline(pasos)

In [ ]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R^2): {r2}')

# Realizar validación cruzada para obtener métricas promedio
mse_scores = -cross_val_score(pipe, X, y, cv=5, scoring='neg_mean_squared_error')
r2_scores = cross_val_score(pipe, X, y, cv=5, scoring='r2')
avg_mse = mse_scores.mean()
avg_r2 = r2_scores.mean()
print(f'Promedio del Error Cuadrático Medio (MSE) con CV: {avg_mse}')
print(f'Promedio del coeficiente de determinación (R^2) con CV: {avg_r2}')


Mean Squared Error (MSE): 152.3531732450157
R-squared (R^2): 0.6806486053468299
Promedio del Error Cuadrático Medio (MSE) con CV: 211.88763784694316
Promedio del coeficiente de determinación (R^2) con CV: 0.43312105272975776


##### Método 4: Knn

In [ ]:
pasos = [
    ('agregar_artistas', AgregarArtistasInvolucrados(selected_features=['artists'])),
    ('encoder', FeatureSelectionFrequencyEncoder(selected_features=['Idioma_Detectado'])),
    ('scaler', StandardScaler()),
    ('KNN', KNeighborsRegressor(n_neighbors=5))
]
pipe = Pipeline(pasos)

In [ ]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R^2): {r2}')

mse_scores = -cross_val_score(pipe, X, y, cv=5, scoring='neg_mean_squared_error')
r2_scores = cross_val_score(pipe, X, y, cv=5, scoring='r2')
avg_mse = mse_scores.mean()
avg_r2 = r2_scores.mean()
print(f'Promedio del Error Cuadrático Medio (MSE) con CV: {avg_mse}')
print(f'Promedio del coeficiente de determinación (R^2) con CV: {avg_r2}')

Mean Squared Error (MSE): 230.38647106638282
R-squared (R^2): 0.5170810080473425
Promedio del Error Cuadrático Medio (MSE) con CV: 319.648832166341
Promedio del coeficiente de determinación (R^2) con CV: 0.0693962210510762




##### Método 5: Random Forest

In [ ]:
pasos = [
    ('agregar_artistas', AgregarArtistasInvolucrados(selected_features=['artists'])),
    ('encoder', FeatureSelectionFrequencyEncoder(selected_features=['Idioma_Detectado'])),
    ('scaler', StandardScaler()),
    ('RandomForest', RandomForestRegressor(n_estimators=100, random_state=42))
]
pipe = Pipeline(pasos)

In [ ]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

# Calcula las métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R^2): {r2}')

# Realiza validación cruzada para obtener métricas promedio
mse_scores = -cross_val_score(pipe, X, y, cv=5, scoring='neg_mean_squared_error')
r2_scores = cross_val_score(pipe, X, y, cv=5, scoring='r2')
avg_mse = mse_scores.mean()
avg_r2 = r2_scores.mean()
print(f'Promedio del Error Cuadrático Medio (MSE) con CV: {avg_mse}')
print(f'Promedio del coeficiente de determinación (R^2) con CV: {avg_r2}')

Mean Squared Error (MSE): 152.3531732450157
R-squared (R^2): 0.6806486053468299
Promedio del Error Cuadrático Medio (MSE) con CV: 211.88763784694316
Promedio del coeficiente de determinación (R^2) con CV: 0.43312105272975776


### PASO 6: Optimizacion de hiperparametros

##### Método 1: GRID SEARCH

*Grid Search es una técnica de optimización de hiperparámetros que explora sistemáticamente un conjunto predefinido de valores para cada hiperparámetro del modelo.*

*Para cada combinación de valores de hiperparámetros en la cuadrícula, se ajusta el modelo y se evalúa su rendimiento mediante validación cruzada u otra métrica de evaluación.*

In [ ]:
pasos = [
    ('agregar_artistas', AgregarArtistasInvolucrados(selected_features=['artists'])),
    ('encoder', FeatureSelectionFrequencyEncoder(selected_features=['Idioma_Detectado'])),
    ('scaler', StandardScaler()),
    ('GradientBoosting', HistGradientBoostingRegressor() )
]
pipe = Pipeline(pasos)

In [ ]:
space=   param_grid = {
    'GradientBoosting__max_iter': [100, 200, 300],  # Número máximo de iteraciones (épocas)
    'GradientBoosting__learning_rate': [0.01, 0.1, 0.2],  # Tasa de aprendizaje
    'GradientBoosting__max_leaf_nodes': [15, 31, 63],  # Número máximo de nodos hoja
    'GradientBoosting__max_depth': [None, 5, 10],  # Profundidad máxima de los árboles
    'GradientBoosting__min_samples_leaf': [1, 2, 4],  # Número mínimo de muestras requeridas en una hoja
    'GradientBoosting__l2_regularization': [0.0, 0.1, 0.2],  # Regularización L2
    'GradientBoosting__verbose': [0],  # Nivel de verbosidad (0 para silencioso)
}

kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
grid_search = GridSearchCV(
    pipe,
    param_grid=space,
    scoring='neg_mean_squared_error',
    cv=kf,
    n_jobs=-1
)


In [ ]:
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mejores hiperparámetros encontrados:")
print(best_params)

print("MSE en el conjunto de prueba:", mse)
print("R^2 en el conjunto de prueba:", r2)

Mejores hiperparámetros encontrados:
{'GradientBoosting__l2_regularization': 0.2, 'GradientBoosting__learning_rate': 0.1, 'GradientBoosting__max_depth': None, 'GradientBoosting__max_iter': 300, 'GradientBoosting__max_leaf_nodes': 63, 'GradientBoosting__min_samples_leaf': 2, 'GradientBoosting__verbose': 0}
MSE en el conjunto de prueba: 151.8576073762491
R^2 en el conjunto de prueba: 0.6816873736767721


##### Método 2: HYPEROPT

*Hyperopt es una biblioteca de optimización de hiperparámetros que utiliza algoritmos de búsqueda más avanzados, como el optimizador de árbol de búsqueda (TPE, Tree-structured Parzen Estimator).*

*Hyperopt optimiza de manera eficiente una función objetivo (generalmente una métrica de rendimiento) utilizando el enfoque probabilístico basado en modelos.*

In [ ]:
param_grid = {
    'GradientBoosting__max_iter': [100, 200, 300],
    'GradientBoosting__learning_rate': [0.01, 0.1, 0.2],
    'GradientBoosting__max_leaf_nodes': [15, 31, 63],
    'GradientBoosting__max_depth': [None, 5, 10],
    'GradientBoosting__min_samples_leaf': [1, 2, 4],
    'GradientBoosting__l2_regularization': [0.0, 0.1, 0.2],
    'GradientBoosting__verbose': [0],

    }

In [ ]:
space = {
    'GradientBoosting__max_iter': hp.choice('max_iter', param_grid['GradientBoosting__max_iter']),
    'GradientBoosting__learning_rate': hp.choice('learning_rate', param_grid['GradientBoosting__learning_rate']),
    'GradientBoosting__max_leaf_nodes': hp.choice('max_leaf_nodes', param_grid['GradientBoosting__max_leaf_nodes']),
    'GradientBoosting__max_depth': hp.choice('max_depth', param_grid['GradientBoosting__max_depth']),
    'GradientBoosting__min_samples_leaf': hp.choice('min_samples_leaf', param_grid['GradientBoosting__min_samples_leaf']),
    'GradientBoosting__l2_regularization': hp.choice('l2_regularization', param_grid['GradientBoosting__l2_regularization']),
    'GradientBoosting__verbose': hp.choice('verbose', [0, 1]),  # Puede ser 0 (silencioso) o 1 (verboso)
}


In [ ]:
def objective(params):
    model = pipe.set_params(**params)
    mse_scores = -cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    return np.mean(mse_scores)

trials = Trials()

best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

print("Mejores hiperparámetros encontrados:")
print(best)
best_model = pipe.set_params(**best_params)

best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

Streaming output truncated to the last 5000 lines.
[35/300] 
1 tree, 63 leaves, max depth = 10, train loss: 72.04801, val loss: 78.61140, in 0.091s
[36/300] 
1 tree, 63 leaves, max depth = 10, train loss: 71.69914, val loss: 78.39187, in 0.068s
[37/300] 
1 tree, 63 leaves, max depth = 10, train loss: 71.38386, val loss: 78.26152, in 0.089s
[38/300] 
1 tree, 63 leaves, max depth = 10, train loss: 71.08228, val loss: 78.16748, in 0.108s
[39/300] 
1 tree, 63 leaves, max depth = 10, train loss: 70.75044, val loss: 77.99683, in 0.082s
[40/300] 
1 tree, 63 leaves, max depth = 10, train loss: 70.48453, val loss: 77.85889, in 0.076s
[41/300] 
1 tree, 63 leaves, max depth = 10, train loss: 70.16524, val loss: 77.72985, in 0.092s
[42/300] 
1 tree, 63 leaves, max depth = 10, train loss: 69.89862, val loss: 77.62029, in 0.056s
[43/300] 
1 tree, 63 leaves, max depth = 10, train loss: 69.59396, val loss: 77.49080, in 0.065s
[44/300] 
1 tree, 63 leaves, max depth = 10, train loss: 69.31623, val loss:

In [ ]:
mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print("Mejores hiperparámetros encontrados:")
print(best_params)

print("MSE en el conjunto de prueba:", mse)
print("R^2 en el conjunto de prueba:", r2)

Mejores hiperparámetros encontrados:
OrderedDict([('GradientBoosting__l2_regularization', 0.2), ('GradientBoosting__learning_rate', 0.1), ('GradientBoosting__max_depth', None), ('GradientBoosting__max_iter', 200), ('GradientBoosting__max_leaf_nodes', 63), ('GradientBoosting__min_samples_leaf', 4), ('GradientBoosting__verbose', 0)])
MSE en el conjunto de prueba: 153.24408369846734
R^2 en el conjunto de prueba: 0.6787811450914185


##### Método 3: Scikit Optimize

*Scikit-Optimize, a menudo abreviado como Skopt, es otra biblioteca de optimización de hiperparámetros que se integra bien con scikit-learn.*

*Utiliza técnicas de optimización secuencial basadas en la búsqueda de acuerdos óptimos para ajustar los hiperparámetros de un modelo. Ofrece métodos Bayesianos y de búsqueda secuencial.*

In [ ]:
space=   param_grid = {
    'GradientBoosting__max_iter': [100, 200, 300],  # Número máximo de iteraciones (épocas)
    'GradientBoosting__learning_rate': [0.01, 0.1, 0.2],  # Tasa de aprendizaje
    'GradientBoosting__max_leaf_nodes': [15, 31, 63],  # Número máximo de nodos hoja
    'GradientBoosting__max_depth': [None, 5, 10],  # Profundidad máxima de los árboles
    'GradientBoosting__min_samples_leaf': [1, 2, 4],  # Número mínimo de muestras requeridas en una hoja
    'GradientBoosting__l2_regularization': [0.0, 0.1, 0.2],  # Regularización L2
    'GradientBoosting__verbose': [0],  # Nivel de verbosidad (0 para silencioso)
}

kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
from skopt import BayesSearchCV
opt = BayesSearchCV(
    pipe,
    space,
    n_iter=50,  # Número de iteraciones de búsqueda
    cv=5,  # Número de divisiones en la validación cruzada
    scoring='neg_mean_squared_error',
    n_jobs=-1
)



Mejores hiperparámetros encontrados:


In [ ]:
opt.fit(X_train, y_train)
best_params = opt.best_params_
best_model = opt.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimir los resultados
print("Mejores hiperparámetros encontrados:")
print(best_params)

print("MSE en el conjunto de prueba:", mse)
print("R^2 en el conjunto de prueba:", r2)

/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist

Mejores hiperparámetros encontrados:
OrderedDict([('GradientBoosting__l2_regularization', 0.2), ('GradientBoosting__learning_rate', 0.1), ('GradientBoosting__max_depth', None), ('GradientBoosting__max_iter', 200), ('GradientBoosting__max_leaf_nodes', 63), ('GradientBoosting__min_samples_leaf', 4), ('GradientBoosting__verbose', 0)])
MSE en el conjunto de prueba: 151.8305967298221
R^2 en el conjunto de prueba: 0.6817439913856338


##### Método 4: Random search

In [ ]:
space=   param_grid = {
    'GradientBoosting__max_iter': [100, 200, 300],  # Número máximo de iteraciones (épocas)
    'GradientBoosting__learning_rate': [0.01, 0.1, 0.2],  # Tasa de aprendizaje
    'GradientBoosting__max_leaf_nodes': [15, 31, 63],  # Número máximo de nodos hoja
    'GradientBoosting__max_depth': [None, 5, 10],  # Profundidad máxima de los árboles
    'GradientBoosting__min_samples_leaf': [1, 2, 4],  # Número mínimo de muestras requeridas en una hoja
    'GradientBoosting__l2_regularization': [0.0, 0.1, 0.2],  # Regularización L2
    'GradientBoosting__verbose': [0],  # Nivel de verbosidad (0 para silencioso)
}

kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
random_search = RandomizedSearchCV(pipe, space, n_iter=50, cv=5, scoring='neg_mean_squared_error', random_state=42)

random_search.fit(X_train, y_train)
best_params = random_search.best_params_
best_estimator = random_search.best_estimator_

y_pred = best_estimator.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R^2): {r2}')

Mean Squared Error (MSE): 154.33508810219368
R-squared (R^2): 0.6764942627739599


In [ ]:
random_search.best_estimator_

Pipeline(steps=[('agregar_artistas',
                 AgregarArtistasInvolucrados(selected_features=['artists'])),
                ('encoder',
                 FeatureSelectionFrequencyEncoder(selected_features=['Idioma_Detectado'])),
                ('scaler', StandardScaler()),
                ('GradientBoosting',
                 HistGradientBoostingRegressor(l2_regularization=0.1,
                                               max_iter=300,
                                               min_samples_leaf=2))])

In [ ]:
random_search.best_params_

{'GradientBoosting__verbose': 0,
 'GradientBoosting__min_samples_leaf': 2,
 'GradientBoosting__max_leaf_nodes': 31,
 'GradientBoosting__max_iter': 300,
 'GradientBoosting__max_depth': None,
 'GradientBoosting__learning_rate': 0.1,
 'GradientBoosting__l2_regularization': 0.1}